In [69]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as kr
import matplotlib.pyplot as plt

from keras.callbacks import EarlyStopping
from sklearn.metrics import ConfusionMatrixDisplay

import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries

In [70]:
img_height = 180
img_width = 180
batch_size = 32

train_ds = kr.utils.image_dataset_from_directory(
    r"C:\Users\pirat\Desktop\cse498r\PNEUMONIA classificatiopn\train",
    seed = 753,
    image_size = (img_height,img_width)
)
test_ds = kr.utils.image_dataset_from_directory(
    r"C:\Users\pirat\Desktop\cse498r\PNEUMONIA classificatiopn\test",
    seed = 753,
    image_size = (img_height,img_width)
)


class_names = test_ds.class_names

Found 392 files belonging to 2 classes.
Found 146 files belonging to 2 classes.


In [71]:
import numpy as np
import tensorflow as tf

def count(counts, batch):
  features, labels = batch
  class_1 = (labels == 1)
  class_1 = tf.cast(class_1, tf.int32)

  class_0 = (labels == 0)
  class_0 = tf.cast(class_0, tf.int32)

  counts['class_0'] += tf.reduce_sum(class_0)
  counts['class_1'] += tf.reduce_sum(class_1)

  return counts

counts = train_ds.take(10).reduce(
    initial_state={'class_0': 0, 'class_1': 0},
    reduce_func=count)

counts = np.array([counts['class_0'].numpy(),
                   counts['class_1'].numpy()]).astype(np.float32)

fractions = counts / counts.sum()
print(fractions)


[0.784375 0.215625]


In [72]:
def class_func(features, label):
  return label

In [73]:
resample_ds = (
    train_ds
    .unbatch()
    .rejection_resample(class_func, target_dist=[0.33,0.33],
                        initial_dist=fractions)
    .batch(32))

balanced_train_ds = resample_ds.map(lambda extra_label, features_and_label: features_and_label)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = balanced_train_ds.cache().prefetch(buffer_size = AUTOTUNE)
val_ds = test_ds.cache().prefetch(buffer_size = AUTOTUNE)

In [74]:
base_model = tf.keras.applications.VGG19(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(img_height, img_width, 3),
    pooling="max",
    classes=1,  # Changed to 1 for binary classification
    classifier_activation="sigmoid"  # Changed to 'sigmoid' for binary classification
)

base_model.trainable = False

In [75]:
mean = np.array([127.5]*3)
var = mean ** 2
norm_layer = kr.layers.experimental.preprocessing.Normalization(mean=mean,variance=var)

data_augmentation = kr.Sequential(
  [
    kr.layers.RandomFlip("horizontal",input_shape=(img_height, img_width, 3)),
    kr.layers.RandomRotation(0.3),
    kr.layers.RandomZoom(-0.1),
    kr.layers.RandomHeight(0.1),
    kr.layers.RandomWidth(0.1)
  ]
)

In [76]:
inputs = kr.Input(shape=(180,180,3))
mean = np.array([127.5]*3)
var = mean ** 2


norm_layer = kr.layers.experimental.preprocessing.Normalization(mean=mean,variance=var)
x = norm_layer(inputs)

x = base_model(x)

x = kr.layers.Dense(256)(x)
x = kr.layers.Dense(64)(x)
output = kr.layers.Dense(3, activation="softmax")(x)
model_fin = kr.Model(inputs,output)

In [77]:
model_fin.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 180, 180, 3)]     0         
                                                                 
 normalization_9 (Normaliza  (None, 180, 180, 3)       0         
 tion)                                                           
                                                                 
 vgg19 (Functional)          (None, 512)               20024384  
                                                                 
 dense_12 (Dense)            (None, 256)               131328    
                                                                 
 dense_13 (Dense)            (None, 64)                16448     
                                                                 
 dense_14 (Dense)            (None, 3)                 195       
                                                           

In [78]:
import tensorflow as tf
import keras as kr

# Assuming you've defined your model 'model_fin' as mentioned in the previous snippet

optimizer = kr.optimizers.Adam(learning_rate=0.0001)

model_fin.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.BinaryCrossentropy(),  # Binary cross-entropy for binary classification
    metrics=['accuracy']
)


In [79]:
history = model_fin.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10,
    shuffle = False,
)

Epoch 1/10


ValueError: in user code:

    File "C:\Users\pirat\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\pirat\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\pirat\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\pirat\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\pirat\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1139, in compute_loss
        return self.compiled_loss(
    File "C:\Users\pirat\anaconda3\Lib\site-packages\keras\src\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, samp